# Spark Accumulators

Spark accumulators are *distributed counters* which allow you to increment a global counter in Python UDFs. This is useful for counting certain events or cases, which are not directly part of your data processing. Best example would be to count broken records.

## Weather Data Example
We will use the weather measurement data again as an example. Instead of using the Spark functions to extract the measurement information, we will write a Python UDF instead. Although this would not be required in our example, this approach might actually be useful in different scenarios. Even the weather data set contains more information which is at non-fixed locations and could not be extracted using simple Spark/SQL string functions.

This example will show to use accumulators to count records. For example this might be useful to count broken records in other examples (weather data does not have broken records, though).

In [ ]:
from pyspark.sql import SparkSession

if not 'spark' in locals():
    spark = SparkSession.builder \
        .master("local[*]") \
        .config("spark.driver.memory","24G") \
        .getOrCreate()

spark

In [ ]:
spark.conf.set("spark.sql.adaptive.enabled", False)
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

# 1 Load Data

As we will not use the previous extraction, we simply load a single year as text data. In the next section we will apply a Python UDF to extract the desired information.

In [ ]:
storageLocation = "s3://dimajix-training/data/weather"

In [ ]:
import pyspark.sql.functions as f

raw_weather = spark.read.text(storageLocation + "/2003").withColumn("year", f.lit(2003))
raw_weather.limit(5).toPandas()

# 2 Extract Weather

We will now create and test a simple Python UDF for extracting the weather data. In the next section we will improve that function for counting invalid USAF and WBAN codes. But step by step...

## 2.1 Define Python UDF

In [ ]:
from pyspark.sql.types import *

schema = StructType([
    StructField("usaf", StringType()),
    StructField("wban", StringType()),
    StructField("air_temperature", FloatType()),
    StructField("air_temperature_qual", IntegerType()),
])

@f.udf(schema)
def extract_weather(row):
    usaf = row[4:10]
    wban = row[10:15]
    air_temperature = float(row[87:92])/10
    air_temperature_qual = int(row[92])
    return (usaf, wban, air_temperature, air_temperature_qual)
    

## 2.2 Use Python UDF

Now we can apply the Python UDF `extract_weather` to process our data.

In [ ]:
result = # YOUR CODE HERE
result.limit(5).toPandas()

### Inspect Schema

Since the UDF returned multiple columns, we now have a nested schema.

In [ ]:
result.printSchema()

# 3 Use Accumulators

As we just said, we want to improve the Python UDF to count certain important events. For example you might be interested at how many records are broken (none in our data set). We chose a different example: We want to count the number of invalid USAF and WBAN.

## 3.1 Create Accumulator

In [ ]:
records_processed = # YOUR CODE HERE
invalid_usaf = # YOUR CODE HERE
invalid_wban = # YOUR CODE HERE

## 3.2 Increment accumulators

Now we need to adopt our Python UDF to increment accumulators on specific events. We want to increment each of the accumulators whenever we process an invalid usaf and/or wban.

In [ ]:
from pyspark.sql.types import *

schema = StructType([
    StructField("usaf", StringType()),
    StructField("wban", StringType()),
    StructField("air_temperature", FloatType()),
    StructField("air_temperature_qual", IntegerType()),
])

@f.udf(schema)
def extract_weather(row):
    usaf = row[4:10]
    wban = row[10:15]
    air_temperature = float(row[87:92])/10
    air_temperature_qual = int(row[92])
    
    # Increment accumulators
    # YOUR CODE HERE
    
    return (usaf, wban, air_temperature, air_temperature_qual)

## 3.3 Execute Query

Now we can use the modified UDF and check if the accumulators are used.

In [ ]:
result = # YOUR CODE HERE
result.limit(5).toPandas()

### Inspect Counters

In [ ]:
print("records_processed=" + str(records_processed.value))
print("invalid_usaf=" + str(invalid_usaf.value))
print("invalid_wban=" + str(invalid_wban.value))

### Force Accumulator Updates

Surprisingly the counters are not increased. Apparently Spark will only propagate accumulators for fully processed partitions. We can enforce this by moving the `limit` operation up as follows:

In [ ]:
limit_result = # YOUR CODE HERE
limit_result.show()

print("records_processed=" + str(records_processed.value))
print("invalid_usaf=" + str(invalid_usaf.value))
print("invalid_wban=" + str(invalid_wban.value))

### Running on all records using `count`

Now let's try to execute the UDF for every record. The method `count()` should do the job

In [ ]:
df = # YOUR CODE HERE
df.show()

print("records_processed=" + str(records_processed.value))
print("invalid_usaf=" + str(invalid_usaf.value))
print("invalid_wban=" + str(invalid_wban.value))

Looking at the execution plan explains that the Spark optimizer actually removed the Python UDF.

In [ ]:
# YOUR CODE HERE

### Running on all records using a `filter` condition

Since that didn't work either, because Spark is too clever, let's force the execution by adding a filter condition which requries the UDF to be executed.

In [ ]:
# YOUR CODE HERE

print("records_processed=" + str(records_processed.value))
print("invalid_usaf=" + str(invalid_usaf.value))
print("invalid_wban=" + str(invalid_wban.value))

### Running again, watching the accumulators

Accumulators won't be reset automatically between query executions. This underlines that accumulators are an *execution metric* and not a purely data dependent result.

In [ ]:
df = result.filter(result["measurement.wban"] != '123').select(f.count("*"))
df.show()

print("records_processed=" + str(records_processed.value))
print("invalid_usaf=" + str(invalid_usaf.value))
print("invalid_wban=" + str(invalid_wban.value))

### Reset Counter

You can also reset counters by simply assign them a value.

In [ ]:
# YOUR CODE HERE

In [ ]:
df = result.filter(result["measurement.wban"] != '123').select(f.count("*"))
df.show()

print("records_processed=" + str(records_processed.value))
print("invalid_usaf=" + str(invalid_usaf.value))
print("invalid_wban=" + str(invalid_wban.value))

## 4 Afterthought

It is important to understand that Spark accumulators actually count how often a specific event was triggered in our Python UDF. Since Spark might evaluate certain code paths multiple times (for example in cases of node failures or in cases when the execution plan executes a certain step multiple times). Therefore accumulators cannot and therefore should not be used for generating statistics over the data itself. But they can be used to understand which code paths have been used more often than others.